In [4]:
import pandas as pd

In [186]:
df = pd.read_csv(r'R:\e2projects_two\SoundCast\Inputs\db_inputs\running_emission_rates_by_veh_type.csv')

In [187]:
# Assumptions matching previous tool development 
# See X:\Trans\AIRQUAL\CMAQ Tool Update 2015\PSRCdata\2025 Emission Factors.xlsx

# Use King County only
df = df[df['county'] == 'king']

# Assume 8 am time period
df = df[df['hourID'] == 8]

In [188]:
speed_bin_lookup = {
    0: 0,
    1: 2.5,
    2: 5,
    3: 10,
    4: 25,
    5: 20,
    6: 25,
    7: 30,
    8: 35,
    9: 40,
    10: 45,
    11: 50,
    12: 55,
    13: 60,
    14: 65,
    15: 70,
    16: 75
}

pollutant_dict = {
    'Summer':
    {
        98: 'CO2e',
        3: 'NOx',
        87: 'VOC'
    },
    'Winter':
    {
        2: 'CO',
        110: 'PM2.5'
    }
}

month_season_dict = {
    'Summer': 7,
    'Winter': 1
}


In [213]:
def produce_rates(df, year, season, road_type):

    results_df = pd.DataFrame()
    df = df[(df['monthID']== month_season_dict[season]) & 
            (df['year']==year) & 
            (df['roadTypeID']==road_type) &
            (df['pollutantID'].isin(pollutant_dict[season].keys()))]
    df = df[df['veh_type'].isin(['light','medium','heavy'])]

    df = df.pivot(index=['avgSpeedBinID','pollutantID'], columns='veh_type', values='ratePerDistance')
    df = df.reset_index()
    df = df.sort_values(['pollutantID','avgSpeedBinID'])
    df['County Group'] = 'King County, Washington'
    df['Year'] = year
    df['Pollutant'] = df['pollutantID'].map(pollutant_dict[season])
    df['Season'] = season
    df['Unit'] = "g/mi"
    df['Speed (mph)'] = df['avgSpeedBinID'].map(speed_bin_lookup)
    df.rename(columns={'light':'LDV','medium':'MDV','heavy':'HDV'}, inplace=True)

    return df

In [214]:
def create_full_table(df, year, season_list, road_type):

    results_df = pd.DataFrame()

    for season in season_list:
        results_df = results_df.append(produce_rates(df, year, season, road_type))

    col_list = ['County Group','Year','Season','Pollutant', 'Unit','Speed (mph)','LDV','MDV','HDV']

    results_df = results_df[col_list]
    results_df = results_df.sort_values(['Pollutant','Speed (mph)'])

    return results_df


# Process and Write Results

In [215]:
df_2018 = create_full_table(df, 2018, ['Summer','Winter'], 5)
df_2018.to_csv('2018_running_rates.csv')

df_2018_idle = create_full_table(df, 2018, ['Summer','Winter'], 1)
df_2018_idle.to_csv('2018_idle_rates.csv')

In [216]:
df_2030 = create_full_table(df, 2030, ['Summer','Winter'], 5)
df_2030.to_csv('2030_running_rates.csv', index=False)

df_2030_idle = create_full_table(df, 2030, ['Summer','Winter'], 1)
df_2030_idle.to_csv('2030_idle_rates.csv', index=False)

In [217]:
df_2040 = create_full_table(df, 2040, ['Summer','Winter'], 5)
df_2040.to_csv('2040_running_rates.csv', index=False)

df_2040_idle = create_full_table(df, 2040, ['Summer','Winter'], 1)
df_2040_idle.to_csv('2040_idle_rates.csv', index=False)